In [1]:
import xarray
import numpy as np
import pandas as pd

In [6]:
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")
country_code['ISO Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Bouvet Island', 'Brazil',
       'Brisith Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands',
       'Colombia', 'Comoros', 'Congo',
       'Congo, Democratic Republic of the', 'Cook Islands', 'Costa Rica',
       "Côte d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       '

In [3]:
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)
crop_class.loc[:,'Item'] = crop_class['Item'].str.lower()
# crop_class['Item'].unique()
crop_class['Item'].unique()

array(['maize (corn)', 'oil palm fruit',
       'anise, badian, coriander, cumin, caraway, fennel and juniper berries, raw',
       'chillies and peppers, dry (capsicum spp., pimenta spp.), raw',
       'chillies and peppers, green (capsicum spp. and pimenta spp.)',
       'cinnamon and cinnamon-tree flowers, raw',
       'cloves (whole stems), raw', 'cocoa beans', 'coffee, green',
       'ginger, raw', 'green garlic', 'hop cones', 'maté leaves',
       'natural rubber in primary forms', 'nutmeg, mace, cardamoms, raw',
       'other stimulant, spice and aromatic crops, n.e.c.',
       'pepper (piper spp.), raw', 'peppermint, spearmint', 'persimmons',
       'pyrethrum, dried flowers', 'tea leaves', 'unmanufactured tobacco',
       'vanilla, raw', 'barley', 'buckwheat', 'canary seed',
       'cereals n.e.c.', 'mixed grain', 'oats', 'rye', 'triticale',
       'abaca, manila hemp, raw', 'agave fibres, raw, n.e.c.',
       'flax, processed but not spun', 'jute, raw or retted',
       'kena

In [2]:
max_year = 4
path_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/states.nc'
luh2_states = xarray.open_dataset(path_LUH2, engine="netcdf4", decode_times=False)

luh2_states_worldwide = luh2_states.isel(time=slice(1120, 1124))
luh2_states_worldwide['time'] = pd.date_range(start="1970-01-01", end="1973-01-01", freq='YS')

c:\Users\HEFRY ANESTI\AppData\Local\Programs\Python\Python39\lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 850-01-01 0:0:0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0850-01-01 0:0:0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [3]:
transition_LUH2 = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/transitions.nc'
luh2_trans = xarray.open_dataset(transition_LUH2, engine="netcdf4", decode_times=False)

luh2_trans_worldwide = luh2_trans.isel(time=slice(1120, 1124))
luh2_trans_worldwide['time'] = pd.date_range(start='1970-01-01', end='1973-01-01', freq='YS')

In [4]:
## add the projection

luh2_added_states_worldwide = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp119-2-1-f_gn_2015-2100.nc",\
                                        engine="netcdf4", decode_times=False)
luh2_added_trans = luh2_added_states_worldwide.drop_vars(['lat_bounds', 'lon_bounds', 'time_bnds'])
luh2_added_trans.coords['time'] = pd.date_range(start='2015-01-01', end='2100-01-01', freq='YS')

static = 'D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/staticData_quarterdeg.nc'
luh2_static = xarray.open_dataset(static, engine="netcdf4")

In [5]:
# ### Convert **ccode** to Country Name

country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code.xlsx", engine="openpyxl")

ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh2_static['ccode'].to_numpy().astype('int64')

In [39]:
ccode_convert = np.zeros((720, 1440), dtype="<U64")
for i in range(720):
    for j in range(1440):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "Unknown"

In [7]:
# # Equation Adaptation
# %%
primf_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
secdf_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-SECDF-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
primn_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-PRIMN-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)
secdn_carbon_density = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis2b_interpolate/CARBON-STOCK-SECDN-INTERPOLATION-WORLDWIDE-1970_2100.nc", engine="netcdf4",)


In [8]:
# Land Clearing
primf_to = ['primf_to_c3ann','primf_to_c4ann','primf_to_c3per','primf_to_c4per','primf_to_c3nfx','primf_to_pastr', 'primf_to_range']
secdf_to = ['secdf_to_c3ann','secdf_to_c4ann','secdf_to_c3per','secdf_to_c4per','secdf_to_c3nfx','secdf_to_pastr', 'secdf_to_range']
primn_to = ['primn_to_c3ann','primn_to_c4ann','primn_to_c3per','primn_to_c4per','primn_to_c3nfx','primn_to_pastr', 'primn_to_range']
secdn_to = ['secdn_to_c3ann','secdn_to_c4ann','secdn_to_c3per','secdn_to_c4per','secdn_to_c3nfx','secdn_to_pastr', 'secdn_to_range']

In [9]:

landuse_class_all = ['primf', 'primn', 'secdf', 'secdn', 'c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx', 'pastr', 'range']
landuse_class_prim_secd = ['primf', 'primn', 'secdf', 'secdn']
landuse_class_pastr_crop = ['c3ann', 'c4ann', 'c3per', 'c4per', 'c3nfx', 'pastr', 'range']
landuse_class_secd = ['secdf','secdn']

carstock_biomass = ['branches', 'leaves', 'stems', 'roots']
carstock_soil = ['litter', 'humus', 'charcoal']

land_clearing_trans = ['primf_to_c3ann','primf_to_c4ann','primf_to_c3per','primf_to_c4per','primf_to_c3nfx','primf_to_pastr', 'primf_to_range',
                       'primn_to_c3ann','primn_to_c4ann','primn_to_c3per','primn_to_c4per','primn_to_c3nfx','primn_to_pastr', 'primn_to_range',
                       'secdf_to_c3ann','secdf_to_c4ann','secdf_to_c3per','secdf_to_c4per','secdf_to_c3nfx','secdf_to_pastr', 'secdf_to_range',
                       'secdn_to_c3ann','secdn_to_c4ann','secdn_to_c3per','secdn_to_c4per','secdn_to_c3nfx','secdn_to_pastr', 'secdn_to_range']

ajl_clearing = ['primf', 'primn', 'secdf', 'secdn']

In [10]:
# Land Clearing
c3ann_col = ["primf_to_c3ann", "secdf_to_c3ann", "primn_to_c3ann", "secdn_to_c3ann"]
c4ann_col = ["primf_to_c4ann", "secdf_to_c4ann", "primn_to_c4ann", "secdn_to_c4ann"]
c3per_col = ["primf_to_c3per", "secdf_to_c3per", "primn_to_c3per", "secdn_to_c3per"]
c4per_col = ["primf_to_c4per", "secdf_to_c4per", "primn_to_c4per", "secdn_to_c4per"]
c3nfx_col = ["primf_to_c3nfx", "secdf_to_c3nfx", "primn_to_c3nfx", "secdn_to_c3nfx"]
pastr_col = ["primf_to_pastr", "secdf_to_pastr", "primn_to_pastr", "secdn_to_pastr"]
range_col = ["primf_to_range", "secdf_to_range", "primn_to_range", "secdn_to_range"]

In [11]:
# %%
carbon_density_biomass = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_density_soil = np.zeros((max_year, 720, 1440), dtype="float32")
carbon_equ_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
carbon_equ_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")

In [12]:
temp_clear_increment_biomass = np.zeros((max_year, 11, 720, 1440), dtype="float32")
temp_clear_increment_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")

In [13]:
temp_carbon_emit_trans_clear_bio = np.zeros((max_year, 56, 720, 1440), dtype="float32")
temp_carbon_emit_trans_clear_soil = np.zeros((max_year, 56, 720, 1440), dtype="float32")

In [14]:
area_land_clearing_bio = np.zeros((max_year, 11, 720, 1440), dtype="float32")
area_land_clearing_soil = np.zeros((max_year, 11, 720, 1440), dtype="float32")

In [15]:
carbon_density_primf_bio = np.zeros((max_year, 720, 1440))
carbon_density_primn_bio = np.zeros((max_year, 720, 1440))
carbon_density_secdf_bio = np.zeros((max_year, 720, 1440))
carbon_density_secdn_bio = np.zeros((max_year, 720, 1440))
carbon_density_crop_c3ann_bio = np.zeros((max_year, 720, 1440))
carbon_density_crop_c4ann_bio = np.zeros((max_year, 720, 1440))
carbon_density_crop_c3per_bio = np.zeros((max_year, 720, 1440))
carbon_density_crop_c4per_bio = np.zeros((max_year, 720, 1440))
carbon_density_crop_c3nfx_bio = np.zeros((max_year, 720, 1440))
carbon_density_range_bio = np.zeros((max_year, 720, 1440))
carbon_density_pastr_bio = np.zeros((max_year, 720, 1440))
carbon_density_all_bio = np.zeros((max_year, 720, 1440))

In [16]:
## Soil
carbon_density_primf_soil = np.zeros((max_year, 720, 1440))
carbon_density_primn_soil = np.zeros((max_year, 720, 1440))
carbon_density_secdf_soil = np.zeros((max_year, 720, 1440))
carbon_density_secdn_soil = np.zeros((max_year, 720, 1440))
carbon_density_crop_c3ann_soil = np.zeros((max_year, 720, 1440))
carbon_density_crop_c4ann_soil = np.zeros((max_year, 720, 1440))
carbon_density_crop_c3per_soil = np.zeros((max_year, 720, 1440))
carbon_density_crop_c4per_soil = np.zeros((max_year, 720, 1440))
carbon_density_crop_c3nfx_soil = np.zeros((max_year, 720, 1440))
carbon_density_range_soil = np.zeros((max_year, 720, 1440))
carbon_density_pastr_soil = np.zeros((max_year, 720, 1440))
carbon_density_all_soil = np.zeros((max_year, 720, 1440))

In [17]:
luc_crops = np.zeros((max_year, 11, 720, 1440), dtype="float32")
luc_pasteur = np.zeros((max_year, 11, 720, 1440), dtype="float32")
# wood_harvest = np.zeros((max_year, 11, 720, 1440), dtype="float32")
# ag_abandon = np.zeros((max_year, 11, 720, 1440), dtype="float32")

In [18]:
# ### Biomass
for time_idx in range(max_year):
    for biomass_class in carstock_biomass:
        carbon_density_primf_bio[time_idx] += primf_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_primn_bio[time_idx] += primn_carbon_density['primn_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_secdf_bio[time_idx] += secdf_carbon_density['secdf_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
        carbon_density_secdn_bio[time_idx] += secdn_carbon_density['secdn_interp'].isel(time=time_idx).sel(NBP=biomass_class).to_numpy()
    
    carbon_density_all_bio[time_idx] += carbon_density_primf_bio[time_idx]+carbon_density_primn_bio[time_idx]+carbon_density_secdf_bio[time_idx]+carbon_density_secdn_bio[time_idx]

# Equation 10
for time_idx in range(max_year):
    for i, col in enumerate(landuse_class_all):
        carbon_equ_biomass[time_idx][i] += luh2_static['carea'].to_numpy()*carbon_density_all_bio[time_idx]*\
                                    np.nan_to_num(luh2_states_worldwide[col].isel(time=time_idx).to_numpy())


In [19]:
# #### Land Clearing

# ##### From
# Equation 10 (Land Clearing and Land Abandonment)
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_prim_secd):
            for k, colk in enumerate(land_clearing_trans):
                if (j == 0):
                    if (k <= 6):
                        temp_carbon_emit_trans_clear_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primf_bio[time_idx]))
                        temp_clear_increment_biomass[time_idx][j] += temp_carbon_emit_trans_clear_bio[time_idx][k]

                elif (j == 1):
                    if (6 < k <= 13):
                        temp_carbon_emit_trans_clear_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primn_bio[time_idx]))
                        temp_clear_increment_biomass[time_idx][j] += temp_carbon_emit_trans_clear_bio[time_idx][k]

                elif (j == 2):
                    if (13 < k <= 20):
                        temp_carbon_emit_trans_clear_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdf_bio[time_idx]))
                        temp_clear_increment_biomass[time_idx][j] += temp_carbon_emit_trans_clear_bio[time_idx][k]

                elif (j == 3):
                    if (20 < k <= 27):
                        temp_carbon_emit_trans_clear_bio[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdn_bio[time_idx]))
                        temp_clear_increment_biomass[time_idx][j] += temp_carbon_emit_trans_clear_bio[time_idx][k]

        carbon_equ_biomass[time_idx] -= temp_clear_increment_biomass[time_idx]


In [20]:
# ##### To

# %%
# Equation 14
for time_idx in range(max_year):
    if (0<time_idx<=max_year-1):
        for i, col in enumerate(landuse_class_pastr_crop):
            if (i == 0):
                for j, c3ann_trans in enumerate(c3ann_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+28] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c3ann_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c3ann_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c3ann_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+28]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]
                
            elif (i == 1):
                for j,c4ann_trans in enumerate(c4ann_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+32] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c4ann_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c4ann_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c4ann_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+32]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]
                
            elif (i == 2):
                for j,c3per_trans in enumerate(c3per_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+36] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c3per_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c3per_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c3per_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+36]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]
                
            elif (i == 3):
                for j,c4per_trans in enumerate(c4per_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+40] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c4per_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c4per_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c4per_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+40]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]
                
            elif (i == 4):
                for j,c3nfx_trans in enumerate(c3nfx_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+44] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c3nfx_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c3nfx_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c3nfx_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+44]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]
                
            elif (i == 5):
                for j,pastr_trans in enumerate(pastr_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+48] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[pastr_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_pastr_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[pastr_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+48]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]
                
            elif (i==6):
                for j,range_trans in enumerate(range_col):
                    temp_carbon_emit_trans_clear_bio[time_idx][j+52] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[range_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_range_bio[time_idx]))
                    area_land_clearing_bio[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[range_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_biomass[time_idx][i+4] += temp_carbon_emit_trans_clear_bio[time_idx][j+52]
                carbon_equ_biomass[time_idx][i+4] += temp_clear_increment_biomass[time_idx][i+4]


In [21]:
# %% [markdown]
# ### Soil
for time_idx in range(max_year):
    for soil_class in carstock_soil:
        carbon_density_primf_soil[time_idx] += primf_carbon_density['primf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_primn_soil[time_idx] += primn_carbon_density['primn_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_secdf_soil[time_idx] += secdf_carbon_density['secdf_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
        carbon_density_secdn_soil[time_idx] += secdn_carbon_density['secdn_interp'].isel(time=time_idx).sel(NBP=soil_class).to_numpy()
    
    carbon_density_all_soil[time_idx] += carbon_density_primf_soil[time_idx]+carbon_density_primn_soil[time_idx]+carbon_density_secdf_soil[time_idx]+carbon_density_secdn_soil[time_idx]

In [22]:
# Equation 10
for time_idx in range(max_year):
    for i, col in enumerate(landuse_class_all):
        carbon_equ_soil[time_idx][i] += luh2_static['carea'].to_numpy()*carbon_density_all_soil[time_idx]*\
                                    np.nan_to_num(luh2_states_worldwide[col].isel(time=time_idx).to_numpy())

In [23]:
# %% [markdown]
# #### Land Clearing

# ##### From
# Equation 10 (Land Clearing and Land Abandonment)
for time_idx in range(max_year):
    if (time_idx<=max_year-2):
        for j, colj in enumerate(landuse_class_prim_secd):
            for k, colk in enumerate(land_clearing_trans):
                if (j == 0):
                    if (k <= 6):
                        temp_carbon_emit_trans_clear_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primf_soil[time_idx]))
                        temp_clear_increment_soil[time_idx][j] += temp_carbon_emit_trans_clear_soil[time_idx][k]

                elif (j == 1):
                    if (6 < k <= 13):
                        temp_carbon_emit_trans_clear_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_primn_soil[time_idx]))
                        temp_clear_increment_soil[time_idx][j] += temp_carbon_emit_trans_clear_soil[time_idx][k]

                elif (j == 2):
                    if (13 < k <= 20):
                        temp_carbon_emit_trans_clear_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdf_soil[time_idx]))
                        temp_clear_increment_soil[time_idx][j] += temp_carbon_emit_trans_clear_soil[time_idx][k]

                elif (j == 3):
                    if (20 < k <= 27):
                        temp_carbon_emit_trans_clear_soil[time_idx][k] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                        np.nan_to_num(luh2_states_worldwide[colk[9:]].isel(time=time_idx).to_numpy()),
                                                                                        carbon_density_secdn_soil[time_idx]))
                        temp_clear_increment_soil[time_idx][j] += temp_carbon_emit_trans_clear_soil[time_idx][k]

        carbon_equ_soil[time_idx] -= temp_clear_increment_soil[time_idx]


In [24]:
for time_idx in range(max_year):
    if (0<time_idx<=max_year-1):
        for i, col in enumerate(landuse_class_pastr_crop):
            if (i == 0):
                for j,c3ann_trans in enumerate(c3ann_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+28] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c3ann_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c3ann_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c3ann_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+28]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]
                
            elif (i == 1):
                for j,c4ann_trans in enumerate(c4ann_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+32] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c4ann_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c4ann_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c4ann_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+32]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]
                
            elif (i == 2):
                for j,c3per_trans in enumerate(c3per_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+36] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c3per_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c3per_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c3per_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+36]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]
                
            elif (i == 3):
                for j,c4per_trans in enumerate(c4per_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+40] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c4per_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c4per_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c4per_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+40]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]
                
            elif (i == 4):
                for j,c3nfx_trans in enumerate(c3nfx_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+44] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[c3nfx_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_crop_c3nfx_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[c3nfx_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+44]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]
                
            elif (i == 5):
                for j,pastr_trans in enumerate(pastr_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+48] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[pastr_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_pastr_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[pastr_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+48]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]
                
            elif (i==6):
                for j,range_trans in enumerate(range_col):
                    temp_carbon_emit_trans_clear_soil[time_idx][j+52] = np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                       np.nan_to_num(luh2_states_worldwide[range_trans[9:]].isel(time=time_idx).to_numpy()),
                                                                                       carbon_density_range_soil[time_idx]))
                    area_land_clearing_soil[time_idx][i+4] += np.multiply.reduce((luh2_static['carea'].to_numpy(),
                                                                                 np.nan_to_num(luh2_states_worldwide[range_trans[9:]].isel(time=time_idx).to_numpy())))
                    temp_clear_increment_soil[time_idx][i+4] += temp_carbon_emit_trans_clear_soil[time_idx][j+52]
                carbon_equ_soil[time_idx][i+4] += temp_clear_increment_soil[time_idx][i+4]


In [25]:
for time_idx in range(max_year):
    luc_crops[time_idx] += temp_clear_increment_biomass[time_idx]+temp_clear_increment_soil[time_idx]
    luc_pasteur[time_idx] += temp_clear_increment_biomass[time_idx]+temp_clear_increment_soil[time_idx]

In [26]:
temp_emission_time = pd.date_range('1970-01-01', '1973-01-01', freq='YS').to_list()
carbon_emission_time = []
luc_crops_worldwide = []
luc_pasteur_worldwide = []

temp_data_emission_crops = []
temp_data_emission_pasteur = []
temp_sum_emission_crops = []
temp_sum_emission_pasteur = []
temp_data_emission_total = []

temp_list_country = []
temp_list_type = []
temp_list_carbon_emission = []
carbon_emission_per_country = pd.DataFrame()

carbon_emission_country_crops_dict = {}
carbon_emission_country_pasteur_dict = {}

In [27]:

for idx in range(len(temp_emission_time)):
    carbon_emission_time.append(temp_emission_time[idx].year)

for time_idx, year in enumerate(carbon_emission_time):
    carbon_emission_country_crops_dict[year] = {}
    carbon_emission_country_pasteur_dict[year] = {}
    temp_data_emission_crops.append([])
    temp_data_emission_pasteur.append([])
    temp_sum_emission_crops.append([])
    temp_sum_emission_pasteur.append([])
    temp_data_emission_total.append([])
    for country_idx, country in enumerate(list(np.unique(ccode_convert))):
        data_emission_crops = 0
        data_emission_pasteur = 0
        data_emission_harvest = 0
        data_emission_abandon = 0
        temp_data_emission_crops[time_idx].append([])
        temp_data_emission_pasteur[time_idx].append([])
        temp_sum_emission_crops[time_idx].append([])
        temp_sum_emission_pasteur[time_idx].append([])
        for j in range(11): # jarak 11
            data_emission_crops += luc_crops[time_idx][j][ccode_convert==country].sum()/1E9
            data_emission_pasteur += luc_pasteur[time_idx][j][ccode_convert==country].sum()/1E9
        temp_sum_emission_crops[time_idx][country_idx].append(data_emission_crops)
        temp_sum_emission_pasteur[time_idx][country_idx].append(data_emission_pasteur)
        carbon_emission_country_crops_dict[year][country] = temp_sum_emission_crops[time_idx][country_idx][0]
        carbon_emission_country_pasteur_dict[year][country] = temp_sum_emission_pasteur[time_idx][country_idx][0]
    
        temp_data_emission_crops[time_idx][country_idx].append([country, "LUC-Crops", carbon_emission_country_crops_dict[year][country]])
        temp_data_emission_pasteur[time_idx][country_idx].append([country, "LUC-Pasteur", carbon_emission_country_pasteur_dict[year][country]])
    
    for country_idx in range(194):
        temp_data_emission_total[time_idx].append(temp_data_emission_crops[time_idx][country_idx])
        temp_data_emission_total[time_idx].append(temp_data_emission_pasteur[time_idx][country_idx])


In [28]:
for i in range(len(temp_data_emission_total[0])):
    temp_list_country.append(temp_data_emission_total[0][i][0][0])
    temp_list_type.append(temp_data_emission_total[0][i][0][1])

for time_idx, year in enumerate(carbon_emission_time):
    temp_list_carbon_emission.append([])
    for record_idx in range(len(temp_data_emission_total[0])):
        temp_list_carbon_emission[time_idx].append(temp_data_emission_total[time_idx][record_idx][0][2])

In [29]:
carbon_emission_per_country_intensity = []

In [30]:
carbon_emission_per_country['Country'] = temp_list_country
carbon_emission_per_country['Type'] = temp_list_type

for time_idx, year in enumerate(carbon_emission_time):
    carbon_emission_per_country_intensity.append(temp_list_carbon_emission[time_idx])
    carbon_emission_per_country.insert(time_idx+2, f'Carbon_Emission_{year}', temp_list_carbon_emission[time_idx], True)


In [31]:
carbon_emission_per_country.to_excel("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION-PER-COUNTRY-1970-2021_vstates.xlsx", index=False)

In [32]:
carbon_emission_per_country

,Country,Type,Carbon_Emission_1970,Carbon_Emission_1971,Carbon_Emission_1972,Carbon_Emission_1973
0,Afghanistan,LUC-Crops,0.199709,0.194388,0.188926,0.0
1,Afghanistan,LUC-Pasteur,0.199709,0.194388,0.188926,0.0
2,Albania,LUC-Crops,0.055271,0.053280,0.051493,0.0
3,Albania,LUC-Pasteur,0.055271,0.053280,0.051493,0.0
4,Algeria,LUC-Crops,0.042605,0.040554,0.037571,0.0
...,...,...,...,...,...,...
383,Yemen,LUC-Pasteur,0.052010,0.051208,0.050418,0.0
384,Zambia,LUC-Crops,1.132415,1.138286,1.144086,0.0
385,Zambia,LUC-Pasteur,1.132415,1.138286,1.144086,0.0
386,Zimbabwe,LUC-Crops,0.546003,0.544881,0.542831,0.0


In [33]:
carbon_emission_per_country_intensity_arr = np.array(carbon_emission_per_country_intensity)
carbon_emission_per_country_intensity_arr.shape

(4, 388)

In [34]:
luh2_states_worldwide['lat'].to_numpy().shape

(720,)

In [35]:
pd.date_range("1970-01-01", "1973-01-01", freq='YS').shape

(4,)

In [36]:
carbon_emission_per_country_intensity_arr.shape

(4, 388)

In [37]:
lonttitude = luh2_states_worldwide['lon'].to_numpy().reshape(4, -1)
lattitude = luh2_states_worldwide['lat'].to_numpy().reshape(4, -1)
lattitude.shape

(4, 180)

In [39]:
# df = xarray.DataArray(carbon_emission_per_country_intensity_arr, coords=[('lon', lonttitude), ('lat', lattitude)])

In [40]:
# carbon_emission_per_country_intensity_netcdf = xarray.Dataset({
#     "carbon_emission":(["time"], carbon_emission_per_country_intensity_arr)
# },
# coords={
#     "time": pd.date_range("1970-01-01", "1973-01-01", freq='YS'),
#     "lon": luh2_states_worldwide['lon'].to_numpy(),
#     "lat": luh2_states_worldwide['lat'].to_numpy()
# })
da  = xarray.DataArray(carbon_emission_per_country_intensity_arr)

da
# da.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION-PER-COUNTRY-1970-1973_vstates.nc", mode='w', format="NETCDF4")

<xarray.DataArray (dim_0: 4, dim_1: 388)>
array([[0.19970884, 0.19970884, 0.05527115, ..., 1.13241464, 0.54600276,
        0.54600276],
       [0.19438813, 0.19438813, 0.05328005, ..., 1.13828582, 0.54488092,
        0.54488092],
       [0.18892616, 0.18892616, 0.0514934 , ..., 1.1440857 , 0.54283096,
        0.54283096],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])
Dimensions without coordinates: dim_0, dim_1

In [ ]:
carbon_emission_per_country.drop(["Country", "Type"], axis=1, inplace=True)


In [ ]:
ds = xarray.Dataset(carbon_emission_per_country)
ds

<xarray.Dataset>
Dimensions:               (dim_0: 388)
Coordinates:
  * dim_0                 (dim_0) int64 0 1 2 3 4 5 ... 382 383 384 385 386 387
Data variables:
    Country               (dim_0) object 'Afghanistan' ... 'Zimbabwe'
    Type                  (dim_0) object 'LUC-Crops' ... 'LUC-Pasteur'
    Carbon_Emission_1970  (dim_0) float64 0.1997 0.1997 0.05527 ... 0.546 0.546
    Carbon_Emission_1971  (dim_0) float64 0.1944 0.1944 ... 0.5449 0.5449
    Carbon_Emission_1972  (dim_0) float64 0.1889 0.1889 ... 0.5428 0.5428
    Carbon_Emission_1973  (dim_0) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [ ]:
ds['country'] = list(np.unique(ccode_convert))
ds['time'] = xarray.DataArray(pd.date_range("1970-01-01", "1973-01-01", freq='YS'), dims=['time'])
ds['lon'] = luh2_states_worldwide.coords["lon"].to_numpy()
ds['lat'] = luh2_states_worldwide.coords["lat"].to_numpy()

In [ ]:
ds

<xarray.Dataset>
Dimensions:               (dim_0: 388, country: 194, time: 4, lon: 1440,
                           lat: 720)
Coordinates:
  * dim_0                 (dim_0) int64 0 1 2 3 4 5 ... 382 383 384 385 386 387
  * country               (country) <U52 'Afghanistan' 'Albania' ... 'Zimbabwe'
  * time                  (time) datetime64[ns] 1970-01-01 ... 1973-01-01
  * lon                   (lon) float64 -179.9 -179.6 -179.4 ... 179.6 179.9
  * lat                   (lat) float64 89.88 89.62 89.38 ... -89.62 -89.88
Data variables:
    Country               (dim_0) object 'Afghanistan' ... 'Zimbabwe'
    Type                  (dim_0) object 'LUC-Crops' ... 'LUC-Pasteur'
    Carbon_Emission_1970  (dim_0) float64 0.1997 0.1997 0.05527 ... 0.546 0.546
    Carbon_Emission_1971  (dim_0) float64 0.1944 0.1944 ... 0.5449 0.5449
    Carbon_Emission_1972  (dim_0) float64 0.1889 0.1889 ... 0.5428 0.5428
    Carbon_Emission_1973  (dim_0) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [ ]:
ds.to_netcdf("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis3_LUC/CARBON_EMISSION-PER-COUNTRY-1970-1973_states.nc", mode='w', format="NETCDF4")